In [1]:
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import GPTNeoForCausalLM, AutoTokenizer
from transformers import RobertaConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-openai-detector")

model = AutoModelForSequenceClassification.from_pretrained("roberta-large-openai-detector")

Some weights of the model checkpoint at roberta-large-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
text=str(input("""Enter the text: """))

Enter the text: with his professional football career on the rise , davis had n't wanted to be shackled with the responsibilities of a baby 


In [54]:
text

"with his professional football career on the rise , davis had n't wanted to be shackled with the responsibilities of a baby "

In [55]:
inputs=tokenizer(text,padding=True, truncation=True, return_tensors="pt")

In [56]:
def text_to_sentences(text):
    clean_text = text.replace('\n', ' ')
    return re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', clean_text)


def chunks_of_2000(text, chunk_size = 2000):
    sentences = text_to_sentences(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk + sentence) <= chunk_size:
            if len(current_chunk)!=0:
                current_chunk += " "+sentence
            else:
                current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    chunks.append(current_chunk)
    return chunks
    
def predict(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
      logits = model(**inputs).logits
    
      probs=logits.softmax(dim=-1)
        

    Ai_generated, Human_Generated = probs.detach().cpu().flatten().numpy().tolist()
    return Ai_generated

def findRealProb(text):
    chunksOfText = (chunks_of_2000(text))
    results = []
    for chunk in chunksOfText:
        output = predict(chunk)
        results.append([output, len(chunk)])
    
    ans = 0
    cnt = 0
    for prob, length in results:
        cnt += length
        ans = ans + prob*length
    realProb = ans/cnt
    return {"Ai_Generated": realProb, "Human_Generated": 1-realProb}, results

In [57]:
findRealProb(text)

({'Ai_Generated': 0.18462973833084106, 'Human_Generated': 0.8153702616691589},
 [[0.18462973833084106, 124]])

In [58]:
def get_ai_percentage(text):
    inputs=tokenizer(text,padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        ai_probs = logits.softmax(dim=-1)
    ai_percentage = ai_probs[0][0] / len(ai_probs)
    return ai_percentage * 100

In [59]:
get_ai_percentage(text)

tensor(18.4630)